In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
import os
os.chdir('gdrive/My Drive/Super_resolution/brainSR')

In [3]:
from data.image_list import create_image_list
from data.dataset import create_dataset
from data.data_loader import create_dataloader
import torch
import time
import random
from collections import OrderedDict
import math
from math import log10
from utils import util, convert, metric
import numpy as np

In [4]:
opt = {
    "name": "SRResNet_x4",
    "model": "sr_resnet",
    "gpu_ids": [0],
    "upscale_factor":4,
    "device": "cuda",

    "datasets":
        {
            "path":
                {
                "root": "data/BrainTumour",
                "folders": ["imagesTr"]
                },
            "train":
                {
                "phase": "train",
                "batch_size": 64,
                "use_shuffle": True,
                "upscale_factor": 4,
                "scale": True,
                "no_of_images": 450,
                "n_workers": 1,
                "gaussian": dict(use=True, sigma=1, kernel_size=3, dim=2)   
                },
            "valid":
                {
                "phase": "valid",
                "no_of_images": 34,
                "scale": True,
                "depth_padding": 1,
                "upscale_factor": 4,
                "gaussian": dict(use=True, sigma=1, kernel_size=3, dim=2)   
                }
    },

    
  "path": 
    {
        "root": ""
    },

  "network":
    {
        "norm_type": "batch",
        "which_model_G": "sr_resnet",
        "ngf": 64,
        "ngb": 16,
        "input_ngc": 1,
        "output_ngc": 1
    },

  "train" : 
    {
        "manual_seed": 400,
        "niter": 50,
        "val_freq": 2,
        "lr_G": 1e-4,
        "criterion": "mse",
        "scheduler": dict(name="ReduceLROnPlateau", factor=0.2, threshold=0.1, patience=5, cooldown=0),
        "scheduler_frequency": 2
    },


    "logger": 
    {
        "print_freq": 150,
        "save_checkpoint_freq": 2
    }
}


In [5]:
def parse(opt_path, is_train=True):
#     with open(opt_path, 'r') as f:
#         opt = json.load(f, object_pairs_hook=OrderedDict)
    opt['is_train'] = is_train

    for key, path in opt['path'].items():
        opt['path'][key] = os.path.expanduser(path)
    if is_train:
        experiments_root = os.path.join(opt['path']['root'], 'experiments', opt['name'])
        opt['path']['experiments_root'] = experiments_root
        opt['path']['options'] = experiments_root
        opt['path']['trained_models'] = os.path.join(experiments_root, 'trained_models')
        opt['path']['log'] = os.path.join(experiments_root, 'log')
    else:
        results_root = os.path.join(opt['path']['root'], 'results', opt['name'])
        opt['path']['results_root'] = results_root
        opt['path']['log'] = os.path.join(results_root, 'log')
        opt['path']['test_images'] = os.path.join(results_root, 'test_images')

    return opt

opt = parse(opt)


In [6]:
# seeding function for reproducibility
def seed_everything(seed):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
SEED = opt["train"]["manual_seed"]
seed_everything(SEED)

In [8]:
training_image_list = create_image_list(opt, train=True)
valid_image_list = create_image_list(opt, valid=True)


0/450 images loaded
50/450 images loaded
100/450 images loaded
150/450 images loaded
200/450 images loaded
250/450 images loaded
300/450 images loaded
350/450 images loaded
400/450 images loaded
0/34 images loaded


In [9]:
training_set = create_dataset(opt["datasets"]["train"], training_image_list)
valid_set = create_dataset(opt["datasets"]["valid"], valid_image_list)


In [10]:
from models.models import create_model
model = create_model(opt)

/content/gdrive/My Drive/Super_resolution/brainSR/models/modules/block.py:150: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(self.C.weight, a=a, mode='fan_in')


---------- Model initialized -------------
Number of parameters in G: 1528724
-----------------------------------------------
Model [SRResNetModel] is created.


In [11]:
from utils.logger import Logger
logger = Logger(opt)

In [12]:
# def validate(val_set, model):
#     print('Start validation phase ...')
#     val_start_time = time.time()
#     model.eval() # Change to eval mode. It is important for BN layers.

#     val_size = len(val_set)

#     val_results = OrderedDict()
#     avg_psnr = 0.0 
#     avg_mse = 0.0
#     for val_data in val_set:
#         depth = val_data['H'].shape[0]
#         mse = 0.0
#         for slice_index in range(depth):
#           val_data_slice = dict(H=val_data['H'][slice_index,:,:,:].unsqueeze(0),
#                                 L=val_data['L'][slice_index,:,:,:].unsqueeze(0))
#           model.feed_data(val_data_slice)
#           model.forward()

#           visuals = model.get_current_visuals()
          
#           sr_img = visuals['super-resolution'] # uint8
#           gt_img = visuals['ground-truth'] # uint8

#           mse_slice = model.criterion(sr_img, gt_img).item()
#           mse += mse_slice

#         mse = mse/depth 
#         psnr = 10 * log10(1/mse)     
#         avg_mse += mse
#         avg_psnr += psnr

#     avg_mse = avg_mse / val_size
#     avg_psnr = avg_psnr / val_size

#     val_results['mse'] = avg_mse
#     val_results['psnr'] = avg_psnr

#     val_duration = time.time() - val_start_time

#     model.train() # Change back to train mode.
#     return val_results, val_duration


In [13]:
def validate_batch(val_set, model, val_batch_no):
    print('Start validation phase ...')
    val_start_time = time.time()
    model.eval() # Change to eval mode. It is important for BN layers.

    val_size = len(val_set)

    val_results = OrderedDict()
    avg_psnr = 0.0 
    avg_mse = 0.0
    for val_data in val_set:
        mse = 0.0

        depth = val_data['H'].shape[0]
        val_batch_size = int(depth/val_batch_no)

        for batch_iteration in range(val_batch_no):
          batch_i_start = val_batch_size*batch_iteration
          batch_i_end = val_batch_size*(batch_iteration+1)
          val_data_slice = dict(H=val_data['H'][batch_i_start:batch_i_end,:,:,:],
                                L=val_data['L'][batch_i_start:batch_i_end,:,:,:])
          model.feed_data(val_data_slice)
          model.forward()

          visuals = model.get_current_visuals()
          
          sr_img = visuals['super-resolution'] # uint8
          gt_img = visuals['ground-truth'] # uint8

          mse_slice = model.criterion(sr_img, gt_img).item()
          mse += mse_slice

        mse = mse/val_batch_no 
        psnr = 10 * log10(1/mse)     
        avg_mse += mse
        avg_psnr += psnr

    avg_mse = avg_mse / val_size
    avg_psnr = avg_psnr / val_size

    val_results['mse'] = avg_mse
    val_results['psnr'] = avg_psnr

    val_duration = time.time() - val_start_time

    model.train() # Change back to train mode.
    return val_results, val_duration


In [14]:
model_dict = torch.load("experiments/SRResNet_x4/trained_models/40_ResNet_with_scheduler.pth")
model_dict.keys()
model.netG.load_state_dict(model_dict['model'])
model.optimizer.load_state_dict(model_dict['optimizer'])
model.scheduler.load_state_dict(model_dict['scheduler'])

In [15]:
model.train()

# run this model
start_time = time.time()

avg_train_loss_list=[]
avg_valid_loss_list=[]
lr_list = []
psnr_list = []
val_batch_no = 16

for iteration in range(41, opt['train']['niter']+41):
  train_loss = 0
  val_loss = 0
  for i, train_data in enumerate(training_set):
    train_start_time = time.time()
    # training
    model.feed_data(train_data)
    model.optimize_parameters(i)
    train_duration = time.time() - train_start_time
    
    train_loss += model.loss.item()

    # print losses
    if i % opt['logger']['print_freq'] == 0:
      losses = model.get_current_losses()
      logger.print_results(losses, iteration, i, train_duration, 'loss')
  avg_train_loss = train_loss/len(training_set)
  avg_train_loss_list.append(avg_train_loss)

  # write training results
  if i == len(training_set)-1:
    train_results = dict(avg_train_loss = avg_train_loss,
                         learning_rate = model.optimizer.state_dict()['param_groups'][0]['lr'])
    
    logger.print_results(train_results, iteration, i, 0, 'loss')

  # validation
  if iteration % opt['train']['val_freq'] == 0 and iteration !=0:
    val_results, val_duration = validate_batch(valid_set, model, val_batch_no)
    print(val_results)
    # Save to log
    logger.print_results(val_results, iteration, i, val_duration, 'val')
    
    avg_valid_loss_list.append(val_results['mse'])
    psnr_list.append(val_results['psnr'])


  # save
  if iteration % opt['logger']['save_checkpoint_freq'] == 0 and iteration !=0:
    print('Saving the model at the end of iteration %d' % (iteration))
    model.save(iteration, network_label='ResNet_with_scheduler')

  # append to lists
  lr = model.optimizer.state_dict()['param_groups'][0]['lr']
  lr_list.append(lr)

  # update learning rate
  if opt['train'].get('scheduler') is not None:
    if iteration % opt['train'].get('scheduler_frequency') == 0 and iteration !=0:
      model.update_learning_rate(val_results['psnr'])

  print('end of iteration ' + str(iteration))

print('lr_list:', lr_list)
print('psnr_list:', psnr_list)
print('avg_train_loss_list', avg_train_loss_list)
print('avg_valid_loss_list', avg_valid_loss_list)

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3121: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


(epoch:  41, iters:        0, time: 1.039) loss: 0.00024997 
(epoch:  41, iters:      150, time: 1.009) loss: 0.00038322 
(epoch:  41, iters:      300, time: 1.017) loss: 0.00052173 
(epoch:  41, iters:      449, time: 0.000) avg_train_loss: 0.00031537 learning_rate: 0.00002000 
end of iteration 41
(epoch:  42, iters:        0, time: 1.011) loss: 0.00020714 
(epoch:  42, iters:      150, time: 1.018) loss: 0.00038765 
(epoch:  42, iters:      300, time: 1.018) loss: 0.00049212 
(epoch:  42, iters:      449, time: 0.000) avg_train_loss: 0.00031176 learning_rate: 0.00002000 
Start validation phase ...
OrderedDict([('mse', 0.00034446483010117264), ('psnr', 34.78428715846184)])
(epoch:  42, iters:      449, time: 187.563) mse: 0.00034446 psnr: 34.78428716 
Saving the model at the end of iteration 42
end of iteration 42
(epoch:  43, iters:        0, time: 1.179) loss: 0.00019129 
(epoch:  43, iters:      150, time: 1.021) loss: 0.00032782 
(epoch:  43, iters:      300, time: 1.008) loss: 0.

RuntimeError: ignored

Load model

In [ ]:
model_dict = torch.load("experiments/SRResNet_x4/trained_models/35_ESPCN_with_scheduler.pth")

In [ ]:
model.netG.load_state_dict(model_dict['model'])

<All keys matched successfully>

In [ ]:
val_results_new, val_duration_new = validate_new(valid_set, model, 16)

Start validation phase ...


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3121: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
